<a href="https://colab.research.google.com/github/EDC21/Deep-Learning/blob/master/Examples%20from%20%E3%80%8AHands-on%20ML%E3%80%8B/Chapter%2011-%20Training%20Deep%20NN/Avoiding_Overfitting_Through_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## $\ell_1$ and $\ell_2$ regularization

In [1]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [4]:
layer = keras.layers.Dense(100, activation="elu",
              kernel_initializer="he_normal",
              kernel_regularizer=keras.regularizers.l2(0.01))

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 1.6089 - accuracy: 0.8148 - val_loss: 0.7397 - val_accuracy: 0.8206
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7180 - accuracy: 0.8279 - val_loss: 0.6806 - val_accuracy: 0.8438


For layers with same activation and initialization, we can use functools.partial() to avoid repeating the same codes.

In [7]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
              activation="elu",
              kernel_initializer="he_normal",
              kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    RegularizedDense(300),
        RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 1.6943 - accuracy: 0.8133 - val_loss: 0.7193 - val_accuracy: 0.8312
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7182 - accuracy: 0.8296 - val_loss: 0.6867 - val_accuracy: 0.8444


#Dropout

Concept: At every training step, every neuron(including the input neurons) has a probability p of being temporarily "dropped out", meaning it will be entirely ignored during this training step, but it may be active during the next step.  

The hyperparametr p is called the dropout rate, and it is typically set between 10~50%: closer to 20~30% in RNN, and closer tp 40~50% in CNN.  

After training, neurons don't get dropped anymore.

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5725 - accuracy: 0.8026 - val_loss: 0.3599 - val_accuracy: 0.8652
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4224 - accuracy: 0.8450 - val_loss: 0.3283 - val_accuracy: 0.8774


#Alpha Dropout

IF we want to regularize a self-normalizing network based on the SELU activation fuction, we should use alpha dropout:a variant of dropout that preserves the mean and standard deviation of its inputs.

In [9]:
tf.random.set_seed(42)
np.random.seed(42)

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6653 - accuracy: 0.7594 - val_loss: 0.5917 - val_accuracy: 0.8416
Epoch 2/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5606 - accuracy: 0.7936 - val_loss: 0.5554 - val_accuracy: 0.8404
Epoch 3/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5283 - accuracy: 0.8054 - val_loss: 0.4850 - val_accuracy: 0.8582
Epoch 4/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5072 - accuracy: 0.8119 - val_loss: 0.4621 - val_accuracy: 0.8604
Epoch 5/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4920 - accuracy: 0.8183 - val_loss: 0.4679 - val_accuracy: 0.8580
Epoch 6/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4871 - accuracy: 0.8190 - val_loss: 0.4740 - val_accuracy: 0.8588
Epoch 7/20
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4718 - accuracy: 0.8246 - val_loss: 0.4656 - val_accuracy:

In [11]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.8669


[0.44206303358078003, 0.8669000267982483]

In [12]:
model.evaluate(X_train_scaled, y_train)

1719/1719 [==============================] - 2s 1ms/step - loss: 0.3284 - accuracy: 0.8893


[0.3283841609954834, 0.8892545700073242]

In [13]:
history = model.fit(X_train_scaled, y_train)

1719/1719 [==============================] - 6s 4ms/step - loss: 0.4161 - accuracy: 0.8452


#Max Norm

In [14]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                    kernel_constraint=keras.constraints.max_norm(1.))

In [17]:
MaxNormDense = partial(keras.layers.Dense,
            activation="selu", kernel_initializer="lecun_normal",
            kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
   keras.layers.Flatten(input_shape=[28,28]),
   MaxNormDense(300),
   MaxNormDense(100),
   keras.layers.Dense(10, activation="softmax")                              
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4745 - accuracy: 0.8329 - val_loss: 0.3773 - val_accuracy: 0.8622
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3532 - accuracy: 0.8717 - val_loss: 0.3818 - val_accuracy: 0.8648
